In [1]:
import numpy as np
import random as rnd
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from IPython.display import display, clear_output
# import ipywidgets.interact class, this class will represent a slide bar.
from ipywidgets import interact
from matplotlib.colors import LinearSegmentedColormap
import plotly.express as px
import pandas as pd


From:

[Landis, Geoffrey A. "The Fermi paradox: an approach based on percolation theory." Journal of the british interplanetary society 51.5 (1998): 163-166](http://www.geoffreylandis.com/percolation.htp)


Space is a 3D grid. each cell of the grid ( a sector) is linked to its 6 immediate Von Numan neighbours. Each sector can be in one of three states:
 
1: Uncolonized
 
2: Colonised by colonising colony
 
3: Colonised by a non colonising colony

Algorithm:
 
Each colonising colony searches its 6 immediate sectors for an uncolonized sector. If one is found then it is colonised with a colonising colony with a probability of p and a a non colonising colony with a probability 1-p



Sector class

This is a unit of 3d space

It contains a state and a link to its 6 imediate neighbours

Attributes

            colonized -> int

            x_pos ->int

            y_pos ->int

            z_pos ->int

            neighbours-> list of Space

            capasity -> boolean # if there are no empty neighbours

Methods

            add_neighbour

            colonize

            get_pos -> x,y,z


In [2]:
class Sector:
    colinising = 0.5
    earth = False
    def __init__(self,x_pos,y_pos,z_pos):
        self.id = str(x_pos)+":"+str(y_pos)+":"+str(z_pos)
        # give slight random stutter in xyz to produce a more 'realistic' plot
        self.x_pos = x_pos + 0.5-rnd.random()
        self.y_pos = y_pos + 0.5-rnd.random()
        self.z_pos = z_pos + 0.5-rnd.random()
        self.neighbours =[]
        self.links=[]
        self.colonized=0
        self.edge = True

    def add_neighbour(self,sector):
        self.neighbours.append(sector)
        self.links.append(sector)

    def colonize(self):
        if len(self.neighbours)>0:
            sector = rnd.choice(self.neighbours)
            if sector.colonized==3:
                Sector.earth=True
            while sector.colonized>0 and len(self.neighbours)>0:
                self.neighbours.remove(sector)
                if len(self.neighbours)>0:
                    sector = rnd.choice(self.neighbours)
                    if sector.colonized==3:
                        Sector.earth=True
                else:
                    return False
            if rnd.random()<Sector.colinising:
                sector.colonized=1
            else:
                sector.colonized=2
            self.neighbours.remove(sector)
            return sector
        else:
             return False

    def get_pos(self):
        c_type= "un colonized"
        if self.colonized == 1:
            c_type = "spacefaring"
        if self.colonized == 2:
            c_type = "non-spacefaring"    
        if self.colonized == 3:
            c_type = "earth"    
        return self.x_pos,self.y_pos,self.z_pos,c_type
    
    def update_neighbours(self):
        self.neighbours =[]
        for l in self.links:
            if l.colonized == 0:
                self.neighbours.append(l)
    
    def swap(self):
        sector=rnd.choice(self.links)
        temp_colonized = self.colonized
        self.colonized = sector.colonized
        self.update_neighbours()
        sector.update_neighbours()
        

        

Space class


Attributes

            size -> int
            
            colonies -> list of Space

            space_faring -> list of Space

            space_chart ->list of np lists (x,y,z,c)


Methods

            init_space

            iterate

            plot


In [3]:

class Space:
    def __init__(self,size) -> None:
        def bounds(v):
            if v>=size: 
                return False
            if v<0:
                return False
            return True
            
        self.size = size
        self.edge = False
        # crate sectors and link in network
        self.space=[]
        for x in range(size):
            self.space.append([])
            for y in range(size):
                self.space[x].append([])
                for z in range(size):
                    self.space[x][y].append(Sector(x,y,z))
        for x in range(1,size-1):
            for y in range(1,size-1):
                for z in range(1,size-1):
                    self.space[x][y][z].edge = False
                    self.space[x][y][z].add_neighbour(self.space[x-1][y][z]) 
                    self.space[x][y][z].add_neighbour(self.space[x+1][y][z]) 
                    self.space[x][y][z].add_neighbour(self.space[x][y-1][z]) 
                    self.space[x][y][z].add_neighbour(self.space[x][y+1][z]) 
                    self.space[x][y][z].add_neighbour(self.space[x][y][z-1]) 
                    self.space[x][y][z].add_neighbour(self.space[x][y][z+1]) 
        
        home_world = self.space[x//2][y//2][z//2] 
        home_world.colonized=1
        self.colonies = [home_world]
        self.space_faring = [home_world]
        x,y,z,c = home_world.get_pos()
        self.star_map = []
        self.star_map.append([x,y,z,c])
        x = rnd.randint(2,self.size-2)
        y = rnd.randint(2,self.size-2)
        z = rnd.randint(2,self.size-2)
        self.earth = self.space[x][y][z]
        self.earth.colonized=3
        self.colonies.append(self.earth)
        self.star_map.append([x,y,z,"earth"])
        
    def drift(self,prob):
        d = int(self.size*self.size*self.size*prob)
        for _ in range(d):
            x = rnd.randint(2,self.size-2)
            y = rnd.randint(2,self.size-2)
            z = rnd.randint(2,self.size-2)
            self.space[x][y][z].swap()

    def iterate(self):
        
        new_worlds = []
        settled = []
        self.drift(0.0)
        for c in self.space_faring:
            new_world = c.colonize()
            if new_world != False:
                self.colonies.append(new_world)
                x,y,z,c_type = new_world.get_pos()
                self.star_map.append([x,y,z,c_type])
                if new_world.colonized==1:
                    new_worlds.append(new_world)
                if new_world.edge:
                    self.edge= True

            if len(c.neighbours)==0:
                settled.append(c)


        for s in settled:
            self.space_faring.remove(s)
        for n in new_worlds:
            self.space_faring.append(n)
        
                    
    

In [9]:
seed = rnd.randrange(1000000)
seed=694350
rnd.seed(seed)

Sector.colinising=0.32
size =50
exp = Space(size)
edge = False
for _ in range(400):
    edge = exp.iterate()
   
df = pd.DataFrame(exp.star_map, columns=['x','y','z','colony_type'])

fig = px.scatter_3d(df, x='x', y='y', z='z', color='colony_type',color_discrete_sequence=["maroon", "green","goldenrod"] ,title="Star map", width=800, height=800)
fig.update_traces(hoverinfo="skip",marker_size =2)
   
fig.update_layout(scene=dict(xaxis_showspikes=False,
                             yaxis_showspikes=False,
                             zaxis_showspikes=False,
                             xaxis = dict(nticks=10, range=[0,size]),
                             yaxis = dict(nticks=10, range=[0,size]),
                             zaxis = dict(nticks=10, range=[0,size]),
                             ))


In [5]:
print("Seed was:", seed)

Seed was: 694350


In [6]:
print(len(exp.star_map)/(size*size*size))

0.368152


In [7]:
print(edge)

None


In [8]:
Sector.earth

False